In [1]:
import mlflow

# Start an MLflow experiment
mlflow.set_experiment('GridSearchCV_Experiment')

/Users/typhaine/Documents/Doc_Gorilla/OpenClassroom--Machine-Learning-Engineer/.venv/lib/python3.11/site-packages/pydantic/_internal/_fields.py:128: UserWarning: Field "model_server_url" has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(
/Users/typhaine/Documents/Doc_Gorilla/OpenClassroom--Machine-Learning-Engineer/.venv/lib/python3.11/site-packages/pydantic/_internal/_config.py:317: UserWarning: Valid config keys have changed in V2:
* 'schema_extra' has been renamed to 'json_schema_extra'
  warnings.warn(message, UserWarning)


<Experiment: artifact_location='file:///Users/typhaine/Documents/Doc_Gorilla/OpenClassroom--Machine-Learning-Engineer/P5/mlruns/992333753801301528', creation_time=1699984116641, experiment_id='992333753801301528', last_update_time=1699984116641, lifecycle_stage='active', name='GridSearchCV_Experiment', tags={}>

In [1]:
import os
import ast
import numpy as np
import pandas as pd


ROOT_PATH = os.getcwd()
DATA_LOAD_FILE = os.path.join(ROOT_PATH, "data/StackOverFlowEmbedded.csv.gz")

df = pd.read_csv(DATA_LOAD_FILE)
#df["BertEmbeddingsMean"] = df["BertEmbeddingsMean"].apply(ast.literal_eval)
#df["BertEmbeddingsSum"] = df["BertEmbeddingsSum"].apply(ast.literal_eval)

#df["FastTextEmbeddings"] = df["FastTextEmbeddings"].apply(ast.literal_eval)
#df["UseEmbeddings"] = df["UseEmbeddings"].apply(ast.literal_eval)
df

,Date,Title,Tags,Score,Label,TitleClean,TitleCleanTokenised,BertEmbeddingsMean,BertEmbeddingsSum
0,2016-02-04 12:49:19,Java contains vs anyMatch behaviour,"java, java-stream, equality",56,java,java contain v anymatch behaviour,"['java', 'contain', 'v', 'anymatch', 'behaviour']","[-0.23472099006175995, -0.24564139544963837, 0...","[-9.154118537902832, -9.5800142288208, 6.10894..."
1,2011-05-04 19:52:46,Getting random numbers in Java,"java, random",557,java,get random number in java,"['get', 'random', 'number', 'in', 'java']","[-0.2763572931289673, -0.42447564005851746, 0....","[-10.777934074401855, -16.554550170898438, 0.3..."
2,2012-03-25 17:38:01,Hibernate generates negative id values when us...,"java, hibernate, jpa, jboss7.x, jpa-2.0",53,java,hibern gener neg id valu when use a sequenc,"['hibern', 'gener', 'neg', 'id', 'valu', 'when...","[-0.349016398191452, -0.19550490379333496, -0....","[-13.611639976501465, -7.624691009521484, -0.3..."
3,2010-08-02 17:35:16,How to get the separate digits of an int number?,"java, integer, modulo",196,java,how to get the separ digit of an int number,"['how', 'to', 'get', 'the', 'separ', 'digit', ...","[-0.25365835428237915, -0.3813159167766571, -0...","[-9.892675399780273, -14.871320724487305, -7.0..."
4,2011-06-10 07:52:16,Eclipse plugin for generating a class diagram,"java, eclipse, plugins, uml, class-diagram",112,java,eclips plugin for gener a class diagram,"['eclips', 'plugin', 'for', 'gener', 'a', 'cla...","[-0.251219242811203, -0.4725949764251709, -0.0...","[-9.797550201416016, -18.431203842163086, -1.6..."
...,...,...,...,...,...,...,...,...,...
9995,2014-06-29 03:25:18,How to call Type Methods within an instance me...,"ios, class, methods, types, swift",61,ios,how to call type method within an instanc method,"['how', 'to', 'call', 'type', 'method', 'withi...","[-0.17384220659732819, -0.3111932873725891, -0...","[-6.953688144683838, -12.447731018066406, -8.0..."
9996,2014-07-31 14:55:53,What is NSLayoutConstraint &quot;UIView-Encaps...,"ios, uitableview, cocoa-touch, autolayout, ios...",306,ios,what is nslayoutconstraint quotuiviewencapsula...,"['what', 'is', 'nslayoutconstraint', 'quotuivi...","[-0.1488007754087448, 0.2860782742500305, 0.34...","[-5.952031135559082, 11.443131446838379, 13.78..."
9997,2015-03-17 20:37:23,How do I fix the xcrun unable to find simctl e...,"ios, xcode, xcrun",359,ios,how do i fix the xcrun unabl to find simctl error,"['how', 'do', 'i', 'fix', 'the', 'xcrun', 'una...","[-0.2691449224948883, -0.272336483001709, 0.01...","[-10.765796661376953, -10.89345932006836, 0.45..."
9998,2013-05-03 17:23:43,NSAttributedString background color and rounde...,"ios, objective-c, uiview, quartz-graphics, nsa...",76,ios,nsattributedstr background color and round corner,"['nsattributedstr', 'background', 'color', 'an...","[-0.3197779357433319, -0.11719034612178802, 0....","[-12.791117668151855, -4.6876139640808105, 7.8..."


In [2]:
df["Title"]

0                     Java contains vs anyMatch behaviour
1                          Getting random numbers in Java
2       Hibernate generates negative id values when us...
3        How to get the separate digits of an int number?
4           Eclipse plugin for generating a class diagram
                              ...                        
9995    How to call Type Methods within an instance me...
9996    What is NSLayoutConstraint &quot;UIView-Encaps...
9997    How do I fix the xcrun unable to find simctl e...
9998    NSAttributedString background color and rounde...
9999    &#39;Project Name&#39; was compiled with optim...
Name: Title, Length: 10000, dtype: object

In [3]:
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier

In [5]:
df["LabelEncoded"] = LabelEncoder().fit_transform(df["Label"].values)
TrainDF, TestDF = train_test_split(df,train_size=0.7, random_state=42, stratify=df["LabelEncoded"].values)

In [5]:

with mlflow.start_run():
    logistic_regression_model = LogisticRegression()
    parameter_grid = {'C': [0.001, 0.01, 0.1, 1, 10, 100], 'penalty': [None, 'l2']}
    grid_search = GridSearchCV(estimator=logistic_regression_model,
                            param_grid=parameter_grid,
                            scoring='accuracy',
                            cv=10)
    grid_search.fit(np.array(TrainDF["BertEmbeddings"].values.tolist()), np.array(TrainDF["LabelEncoded"].values.tolist()))

    best_model = grid_search.best_estimator_
    best_params = grid_search.best_params_

    for param_name in best_params:
        mlflow.log_param(param_name, best_params[param_name])
    mlflow.log_metric("best_score", grid_search.best_score_)

    y_pred = best_model.predict(np.array(TestDF["BertEmbeddings"].values.tolist()))
    accuracy = accuracy_score(np.array(TestDF["LabelEncoded"].values.tolist()), y_pred)
    print(f"Accuracy of the best model: {accuracy}")

    # Log the best model
    mlflow.sklearn.log_model(grid_search.best_estimator_, "best_model")

    # End the MLflow run
    mlflow.end_run()

/Users/typhaine/Documents/Doc_Gorilla/OpenClassroom--Machine-Learning-Engineer/.venv/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:1193: UserWarning: Setting penalty=None will ignore the C and l1_ratio parameters
  warnings.warn(
/Users/typhaine/Documents/Doc_Gorilla/OpenClassroom--Machine-Learning-Engineer/.venv/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:1193: UserWarning: Setting penalty=None will ignore the C and l1_ratio parameters
  warnings.warn(
/Users/typhaine/Documents/Doc_Gorilla/OpenClassroom--Machine-Learning-Engineer/.venv/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:1193: UserWarning: Setting penalty=None will ignore the C and l1_ratio parameters
  warnings.warn(
/Users/typhaine/Documents/Doc_Gorilla/OpenClassroom--Machine-Learning-Engineer/.venv/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:1193: UserWarning: Setting penalty=None will ignore the C and l1_ratio parameters
  warnings.warn(
/Users/typha

Accuracy of the best model: 0.805


In [7]:
#!mlflow ui

In [7]:
random_forest_model = RandomForestClassifier()
parameter_grid = {
    'n_estimators': [200, 300, 400, 500, 1000],
    'max_depth': [None],
    #'min_samples_split': [2, 5, 10],
    #'min_samples_leaf': [1, 2, 4]
}
grid_search = GridSearchCV(estimator=random_forest_model,
                           param_grid=parameter_grid,
                           scoring='accuracy',
                           cv=10,
                           n_jobs=-1, verbose=2)  # n_jobs=-1 to use all processors
grid_search.fit(np.array(TrainDF["BertEmbeddingsSum"].values.tolist()), np.array(TrainDF["LabelEncoded"].values.tolist()))

best_model = grid_search.best_estimator_
y_pred = best_model.predict(np.array(TestDF["BertEmbeddingsSum"].values.tolist()))
accuracy = accuracy_score(np.array(TestDF["LabelEncoded"].values.tolist()), y_pred)
print(f"Accuracy of the best model: {accuracy}")

Fitting 10 folds for each of 5 candidates, totalling 50 fits
[CV] END ...................max_depth=None, n_estimators=200; total time= 1.6min
[CV] END ...................max_depth=None, n_estimators=200; total time= 1.6min
[CV] END ...................max_depth=None, n_estimators=200; total time= 1.6min
[CV] END ...................max_depth=None, n_estimators=200; total time= 1.6min
[CV] END ...................max_depth=None, n_estimators=200; total time= 2.0min
[CV] END ...................max_depth=None, n_estimators=200; total time= 2.0min
[CV] END ...................max_depth=None, n_estimators=200; total time= 2.0min
[CV] END ...................max_depth=None, n_estimators=200; total time= 2.0min
[CV] END ...................max_depth=None, n_estimators=200; total time= 1.8min
[CV] END ...................max_depth=None, n_estimators=200; total time= 1.8min
[CV] END ...................max_depth=None, n_estimators=300; total time= 2.7min
[CV] END ...................max_depth=None, n_es

In [17]:
decision_tree_model = DecisionTreeClassifier()
parameter_grid = {
    'criterion': ['gini', 'entropy'],
    'splitter': ['best', 'random'],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}
grid_search = GridSearchCV(estimator=decision_tree_model,
                           param_grid=parameter_grid,
                           scoring='accuracy',
                           cv=10,
                           n_jobs=-1)  # n_jobs=-1 to use all processors
grid_search.fit(np.array(TrainDF["BertEmbeddings"].values.tolist()), np.array(TrainDF["LabelEncoded"].values.tolist()))

best_model = grid_search.best_estimator_
y_pred = best_model.predict(np.array(TestDF["BertEmbeddings"].values.tolist()))
accuracy = accuracy_score(np.array(TestDF["LabelEncoded"].values.tolist()), y_pred)
print(f"Accuracy of the best model: {accuracy}")

Accuracy of the best model: 0.645
